Entrenamiento modelo

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, Subset
import numpy as np

# Configuración del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformaciones para los datos
data_transforms = {
    'Train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Validation': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Carga de datos
data_dir = r"C:\Users\capor\Downloads\calvos\Dataset"
datasets_dict = {x: datasets.ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x]) for x in ['Train', 'Validation', 'Test']}

# Función para reducir la clase mayoritaria en un 96%. Así hago el shuffle y me aseguro que no siga patrones dentro de las imágenes
def balance_dataset(dataset, reduction_ratio=0.96):
    indices_bald = [i for i, (_, label) in enumerate(dataset) if label == 0]  # Clase Bald
    indices_notbald = [i for i, (_, label) in enumerate(dataset) if label == 1]  # Clase NotBald
    
    reduced_notbald_size = int(len(indices_notbald) * (1 - reduction_ratio))
    np.random.seed(42)  # Para reproducibilidad
    selected_notbald = np.random.choice(indices_notbald, reduced_notbald_size, replace=False)
    
    balanced_indices = indices_bald + list(selected_notbald)
    np.random.shuffle(balanced_indices)
    
    return Subset(dataset, balanced_indices)

# Reducir la clase NotBald en Train, Validation y Test
datasets_dict['Train'] = balance_dataset(datasets_dict['Train'])
datasets_dict['Validation'] = balance_dataset(datasets_dict['Validation'])
datasets_dict['Test'] = balance_dataset(datasets_dict['Test'])

# Crear DataLoaders
dataloaders = {x: DataLoader(datasets_dict[x], batch_size=32, shuffle=True, num_workers=4) for x in ['Train', 'Validation', 'Test']}

# Clases en el dataset
class_names = datasets_dict['Train'].dataset.classes
print(f'Clases: {class_names}')

# Cargar modelo preentrenado
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Clasificación binaria
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Función de entrenamiento
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        # Entrenamiento
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in dataloaders['Train']:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Train Loss: {running_loss/len(dataloaders["Train"]):.4f}, Accuracy: {100 * correct/total:.2f}%')
        
        # Validación
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in dataloaders['Validation']:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print(f'Val Loss: {val_loss/len(dataloaders["Validation"]):.4f}, Accuracy: {100 * correct/total:.2f}%')
    
    return model

# Entrenar modelo
model = train_model(model, dataloaders, criterion, optimizer, num_epochs=6)

# Evaluación
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Test Accuracy: {100 * correct/total:.2f}%')

evaluate_model(model, dataloaders['Test'])


Clases: ['Bald', 'NotBald']


c:\Users\capor\.conda\envs\torch_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\capor\.conda\envs\torch_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/6
Train Loss: 0.2578, Accuracy: 88.86%
Val Loss: 0.1897, Accuracy: 92.47%
Epoch 2/6
Train Loss: 0.1927, Accuracy: 92.49%
Val Loss: 0.1476, Accuracy: 93.87%
Epoch 3/6
Train Loss: 0.1796, Accuracy: 93.08%
Val Loss: 0.1565, Accuracy: 93.80%
Epoch 4/6
Train Loss: 0.1866, Accuracy: 92.64%
Val Loss: 0.1428, Accuracy: 94.10%
Epoch 5/6
Train Loss: 0.1922, Accuracy: 92.55%
Val Loss: 0.1759, Accuracy: 92.77%
Epoch 6/6
Train Loss: 0.1809, Accuracy: 92.66%
Val Loss: 0.1312, Accuracy: 94.54%
Test Accuracy: 94.60%


In [3]:
torch.save(model.state_dict(), 'modelo_calvicie2.pth')

In [35]:
ruta_guardado = r"C:\Users\capor\Downloads\modelo_calvicie.pth"
torch.save(model.state_dict(), ruta_guardado)
print(f"Modelo guardado en {ruta_guardado}")


Modelo guardado en C:\Users\capor\Downloads\modelo_calvicie.pth


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import torch.nn as nn

# Cargar el modelo entrenado
device = torch.device("cpu")  # Usar CPU explícitamente
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Bald vs Not Bald
model.load_state_dict(torch.load(r"C:\Users\capor\Downloads\modelo_calvicie.pth", map_location=device))
model = model.to(device)
model.eval()

# Transformaciones de la imagen
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Cargar la imagen de prueba para ver funcionamiento
image = Image.open(r"C:\Users\capor\Downloads\gettyimages-1404205960-612x612.jpg")

# Preprocesar la imagen
image = transform(image).unsqueeze(0).to(device)

# Hacer la predicción
with torch.no_grad():
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    class_names = ["Bald", "Not Bald"]
    result = class_names[predicted.item()]

print(f"Predicción: {result}")

Predicción: Not Bald


C:\Users\capor\AppData\Local\Temp\ipykernel_6672\1650679462.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"C:\Users\capor\Downloads\